In [1]:
import numpy as np
import dask
print('dask', dask.__version__)
import dask.array as da
import h5py
print('h5py', h5py.__version__)
import bcolz
print('bcolz', bcolz.__version__)
import sys
sys.path.insert(0, '../..')
%reload_ext autoreload
%autoreload 1
%aimport allel.model.dask
%aimport allel.model.chunked
%aimport allel.model.ndarray
%aimport allel.model
%aimport allel
print('scikit-allel', allel.__version__)

dask 0.7.5
h5py 2.5.0
bcolz 0.12.1
scikit-allel 0.21.0.dev0


In [2]:
callset = h5py.File('/data/coluzzi/ag1000g/data/phase1/release/AR3/variation/main/hdf5/ag1000g.phase1.ar3.pass.h5', mode='r')
callset

<HDF5 file "ag1000g.phase1.ar3.pass.h5" (mode r)>

In [3]:
genotype = allel.model.chunked.GenotypeChunkedArray(callset['3L/calldata/genotype'])
genotype

,0,1,2,3,4,...,760,761,762,763,764
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0


In [4]:
g = genotype.copy(stop=2000000)
g

,0,1,2,3,4,...,760,761,762,763,764
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0


In [10]:
g.data.chunklen * 765 * 2

2096100

In [12]:
for n in 1, 2, 4, 8:
    bcolz.blosc_set_nthreads(n)
    print('\nblosc threads:', n)
    %time g.max()


threads: 1
CPU times: user 3.64 s, sys: 0 ns, total: 3.64 s
Wall time: 3.64 s

threads: 2
CPU times: user 4.04 s, sys: 52 ms, total: 4.1 s
Wall time: 3.54 s

threads: 4
CPU times: user 4.43 s, sys: 80 ms, total: 4.51 s
Wall time: 3.36 s

threads: 8
CPU times: user 4.66 s, sys: 164 ms, total: 4.83 s
Wall time: 3.25 s


In [22]:
def toarray(x):
    a = np.asarray(x)
    print(a.nbytes)

In [25]:
for n in 1, 2, 4, 8:
    bcolz.blosc_set_nthreads(n)
    print('\nblosc threads:', n)
    %time toarray(g)


threads: 1
3060000000
CPU times: user 1.06 s, sys: 172 ms, total: 1.23 s
Wall time: 1.24 s

threads: 2
3060000000
CPU times: user 1.32 s, sys: 212 ms, total: 1.53 s
Wall time: 972 ms

threads: 4
3060000000
CPU times: user 1.59 s, sys: 200 ms, total: 1.79 s
Wall time: 786 ms

threads: 8
3060000000
CPU times: user 1.74 s, sys: 276 ms, total: 2.02 s
Wall time: 748 ms


In [29]:
def toarraymax(x):
    a = np.asarray(x)
    print(a.max())

In [30]:
for n in 1, 2, 4, 8:
    bcolz.blosc_set_nthreads(n)
    print('\nthreads:', n)
    %time toarraymax(g)


threads: 1
3
CPU times: user 3.52 s, sys: 164 ms, total: 3.68 s
Wall time: 3.67 s

threads: 2
3
CPU times: user 3.73 s, sys: 224 ms, total: 3.95 s
Wall time: 3.4 s

threads: 4
3
CPU times: user 3.98 s, sys: 208 ms, total: 4.19 s
Wall time: 3.22 s

threads: 8
3
CPU times: user 4.15 s, sys: 316 ms, total: 4.46 s
Wall time: 3.18 s


In [31]:
x = np.asarray(g)

In [32]:
%time x.max()

CPU times: user 2.47 s, sys: 0 ns, total: 2.47 s
Wall time: 2.45 s


3

In [26]:
gd = allel.model.dask.GenotypeDaskArray.from_array(g)
gd

,0,1,2,3,4,...,760,761,762,763,764
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0


In [35]:
for n in 1, 2, 4, 8:
    bcolz.blosc_set_nthreads(n)
    for m in 1, 2, 4, 8:
        print('\nblosc threads:', n, '; dask threads:', m)
        %time gd.max().compute(num_workers=m)


blosc threads: 1 ; dask threads: 1
CPU times: user 4.2 s, sys: 1.31 s, total: 5.52 s
Wall time: 5.35 s

blosc threads: 1 ; dask threads: 2
CPU times: user 4.73 s, sys: 876 ms, total: 5.6 s
Wall time: 3.45 s

blosc threads: 1 ; dask threads: 4
CPU times: user 5.08 s, sys: 1.23 s, total: 6.31 s
Wall time: 2.66 s

blosc threads: 1 ; dask threads: 8
CPU times: user 5.17 s, sys: 1.28 s, total: 6.45 s
Wall time: 2.63 s

blosc threads: 2 ; dask threads: 1
CPU times: user 4.61 s, sys: 1.45 s, total: 6.06 s
Wall time: 5.03 s

blosc threads: 2 ; dask threads: 2
CPU times: user 4.95 s, sys: 1.65 s, total: 6.6 s
Wall time: 3.23 s

blosc threads: 2 ; dask threads: 4
CPU times: user 5.29 s, sys: 996 ms, total: 6.28 s
Wall time: 2.01 s

blosc threads: 2 ; dask threads: 8
CPU times: user 5.25 s, sys: 1.66 s, total: 6.9 s
Wall time: 2.08 s

blosc threads: 4 ; dask threads: 1
CPU times: user 5.03 s, sys: 1.52 s, total: 6.54 s
Wall time: 4.75 s

blosc threads: 4 ; dask threads: 2
CPU times: user 5.23 s,